## Description: This is International caller intent predictive model. This model has 4 predictives : billing, plan_travelpass,phone_settings,cruise

## Import all the required libraries

In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.ml.feature import *
import pandas
from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.classification import  RandomForestClassifier,RandomForestClassificationModel
import numpy as np
from pyspark.ml.linalg import *
from pyspark.ml.linalg import Vectors, VectorUDT

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.functions import *
from pyspark.sql.types import *
from sklearn.metrics import confusion_matrix

from pyspark.sql.functions import concat, col, lit

from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
data_all = spark.sql("select  mtn,ivrcallid, day(end_tm_local) AS process_date from vzw_cja_prd_tbls.verint_call_transcripts_raw where day(end_tm_local)>14 AND month(end_tm_local)=11 AND year(end_tm_local)=2019")
driverDF_all= spark.sql("SELECT ivr_id AS ivrcallid, primary_topic as primary,secondary_topic, day(START_TM_LOCAL) AS process_date FROM vzw_soi_prd_tbls.vzsoi_intl_calltopics_v1 where day(START_TM_LOCAL)>14 AND month(START_TM_LOCAL)=11 AND year(START_TM_LOCAL)=2019")
joinDF= data_all.join(driverDF_all,on=['ivrcallid','process_date'],how='inner').dropDuplicates(['ivrcallid'])

In [ ]:
joinDF.dtypes

In [ ]:
split_col = split(joinDF['mtn'], '-')
joinDF = joinDF.withColumn('mtn1', split_col.getItem(0))
joinDF = joinDF.withColumn('mtn2', split_col.getItem(1))
joinDF = joinDF.withColumn('mtn3',split_col.getItem(2))
topictable=joinDF.withColumn("mtnconcat",concat(col("mtn1"), lit(""), col("mtn2"),lit(""),col("mtn3")))

## Load daily batch data and perform feature transformation

In [ ]:
### Load customers who travelled_atleast_once and bill_curr_amt is not null############
features_all = spark.sql("select  mtn as mtnconcat,to_date(process_dt) as call_start_tm_stmp,day(to_date(process_dt)) as process_date, * from VZW_SOI_PRD_TBLS.vzsoi_intl_dly_scoring_batch_tbl where day(to_date(process_dt))>14 AND month(to_date(process_dt))=11 AND year(to_date(process_dt))=2019")

In [ ]:
ValidationDF= features_all.join(topictable,on=['mtnconcat','process_date'],how='inner')

In [ ]:
ValidationDF.count()

In [ ]:
features_all.show(10)

In [ ]:
features = ValidationDF.select('mtnconcat','call_start_tm_stmp','tenure','pplan_eff_dt',
                            'prepaid_ind','access_amt','data_access_amt','feat_amt','late_pymnt_chrg_amt',
                            'occ_amt','bill_6_mth_avg','bill_curr_amt','msg_allow_share_ind','data_allow_share_ind',
                            'voice_allow_share_ind', 'bill_cycle_dt','free_plan','ccd1','call_ts1',
                            'tvl1','cl_desc1','primary1_topic','primary2_topic','primary3_topic','primary4_topic',
                            'primary5_topic','line_type','tvl_pass_included','can_mex_free_plan','prev_tvl_count',
                            'prev_call_count',"plans_visited_ind",
                            "intl_plans_visited_ind",
                            "trvln_outsid_us_ind",
                            "trip_planner_ind","while_outside_us_ind","support_ind","billing_and_payment_ind",
                            "plan_and_account_ind","device_troubleshooting_assistant_ind","international_services_ind",
                            "bill_ind","overview_ind","next_bill_estimate_ind","payment_history_ind","family_controls_ind",
                            "mfapp_plans_visited_ind","mfapp_intl_plans_visited_ind","mfapp_trvln_outsid_us_ind",
                            "mfapp_trip_planner_ind","mfapp_while_outside_us_ind","mfapp_support_ind","mfapp_billing_and_payment_ind",
                            "mfapp_plan_and_account_ind","mfapp_device_troubleshooting_assistant_ind","mfapp_international_services_ind",
                            "mfapp_bill_ind","mfapp_overview_ind","mfapp_next_bill_estimate_ind","mfapp_payment_history_ind",
                            "mfapp_family_controls_ind","process_dt","process_date","primary")

In [ ]:
#features.dtypes

In [ ]:
features=features.filter(features.prev_tvl_count!=0)

In [ ]:
#features = features.withColumn('age', features['age'].cast(DoubleType()))
features = features.withColumn('prev_tvl_count', features['prev_tvl_count'].cast(DoubleType()))
features = features.withColumn('prev_call_count', features['prev_call_count'].cast(DoubleType()))
features = features.withColumn('access_amt', features['access_amt'].cast(FloatType()))
features = features.withColumn('data_access_amt', features['data_access_amt'].cast(FloatType()))
features = features.withColumn('feat_amt', features['feat_amt'].cast(FloatType()))
features = features.withColumn('late_pymnt_chrg_amt', features['late_pymnt_chrg_amt'].cast(FloatType()))
features = features.withColumn('occ_amt', features['occ_amt'].cast(FloatType()))
features = features.withColumn('bill_6_mth_avg', features['bill_6_mth_avg'].cast(FloatType()))
features = features.withColumn('bill_curr_amt', features['bill_curr_amt'].cast(FloatType()))

In [ ]:
features = features.withColumn('bill_6_mth_avg', when(features['bill_6_mth_avg'].isNull(), 0).otherwise(col('bill_6_mth_avg')))
features = features.withColumn('bill_curr_amt', when(features['bill_curr_amt'].isNull(), 0).otherwise(col('bill_curr_amt')))
features = features.withColumn('prev_tvl_count', when(features['prev_tvl_count'].isNull(), 0).otherwise(col('prev_tvl_count')))
features = features.withColumn('prev_call_count', when(features['prev_call_count'].isNull(), 0).otherwise(col('prev_call_count')))

In [ ]:
features=features.withColumn('free_plan',lower(features['free_plan']))
features = features.withColumn('free_plan', when(features['free_plan'].isNull(), 'paid').otherwise(col('free_plan')))

In [ ]:
features=features.withColumn('access_data_diff',features['access_amt']-features['data_access_amt'])
features=features.withColumn('bill_shock',(features['bill_curr_amt']-features['bill_6_mth_avg'])/features['bill_6_mth_avg'])

In [ ]:
####Replaced with median values from training dataset
features = features.withColumn('access_amt', when(features['access_amt'].isNull(), 35).otherwise(col('access_amt')))
features = features.withColumn('data_access_amt', when(features['data_access_amt'].isNull(), 35).otherwise(col('data_access_amt')))
features = features.withColumn('feat_amt', when(features['feat_amt'].isNull(), 10).otherwise(col('feat_amt')))
features = features.withColumn('late_pymnt_chrg_amt', when(features['late_pymnt_chrg_amt'].isNull(), 0).otherwise(col('late_pymnt_chrg_amt')))
features = features.withColumn('occ_amt', when(features['occ_amt'].isNull(), 0).otherwise(col('occ_amt')))
features = features.withColumn('bill_6_mth_avg', when(features['bill_6_mth_avg'].isNull(), 204.53).otherwise(col('bill_6_mth_avg')))
features = features.withColumn('bill_curr_amt', when(features['bill_curr_amt'].isNull(), 230.5).otherwise(col('bill_curr_amt')))
features = features.withColumn('prev_tvl_count', when(features['prev_tvl_count'].isNull(), 1).otherwise(col('prev_tvl_count')))
features = features.withColumn('prev_call_count', when(features['prev_call_count'].isNull(), 0).otherwise(col('prev_call_count')))
features = features.withColumn('access_data_diff', when(features['access_data_diff'].isNull(),0).otherwise(col('access_data_diff')))
features = features.withColumn('bill_shock', when(features['bill_shock'].isNull(), 0.033).otherwise(col('bill_shock')))

In [ ]:
features=features.withColumn('tvl1_call1',datediff('call_start_tm_stmp','tvl1'))
features=features.withColumn('bill_call1',datediff('call_start_tm_stmp','bill_cycle_dt'))

In [ ]:
features = features.withColumn('tvl1_call1', when(features['tvl1_call1'].isNull(), 60).otherwise(col('tvl1_call1')))
features = features.withColumn('bill_call1', when(features['bill_call1'].isNull(), 60).otherwise(col('bill_call1')))

In [ ]:
features = features.withColumn('msg_allow_share_ind', when(features['msg_allow_share_ind'].isNull(), 'Unknown').otherwise(col('msg_allow_share_ind')))
features = features.withColumn('data_allow_share_ind', when(features['data_allow_share_ind'].isNull(), 'Unknown').otherwise(col('data_allow_share_ind')))
features = features.withColumn('voice_allow_share_ind', when(features['voice_allow_share_ind'].isNull(), 'Unknown').otherwise(col('voice_allow_share_ind')))
features = features.withColumn('ccd1', when(features['ccd1'].isNull(), 'Unknown').otherwise(col('ccd1')))
features = features.withColumn('free_plan', when(features['free_plan'].isNull(), 'Unknown').otherwise(col('free_plan')))
features = features.withColumn('cl_desc1', when(features['cl_desc1'].isNull(), 'Unknown').otherwise(col('cl_desc1')))
features = features.withColumn('line_type', when(features['line_type'].isNull(), 'Unknown').otherwise(col('line_type')))
features = features.withColumn('tvl_pass_included', when(features['tvl_pass_included'].isNull(), 'Unknown').otherwise(col('tvl_pass_included')))
features = features.withColumn('can_mex_free_plan', when(features['can_mex_free_plan'].isNull(), 'Unknown').otherwise(col('can_mex_free_plan')))

In [ ]:
features = features.withColumn('ccd1', when((col('ccd1') =='BHS')| (col('ccd1') =='DOM')|(col('ccd1') =='BRB')|(col('ccd1') =='LCA')| (col('ccd1') =='VGB')|(col('ccd1') =='TCA')|(col('ccd1') =='CUR')|(col('ccd1') =='GRD')|(col('ccd1') =='ATG')|(col('ccd1') =='BMU')|(col('ccd1') =='JAM')| (col('ccd1') =='CRI')| (col('ccd1') =='CUB')| (col('ccd1') =='KNA')|(col('ccd1') =='TTO')|(col('ccd1') =='HTI')| (col('ccd1') =='ABW'),'CARR').otherwise(col('ccd1')))
features = features.withColumn('ccd1', when((col('ccd1') =='ITA')| (col('ccd1') =='HUN')| (col('ccd1') =='DNK')| (col('ccd1') =='NOR')| (col('ccd1') =='CZE')| (col('ccd1') =='BEL')| (col('ccd1') =='ISL')| (col('ccd1') =='PRT')| (col('ccd1') =='POL')| (col('ccd1') =='FRA')| (col('ccd1') =='DEU')| (col('ccd1') =='NLD')| (col('ccd1') =='ESP')| (col('ccd1') =='AUT')| (col('ccd1') =='CHE'),'SCH').otherwise(col('ccd1')))
features = features.withColumn('ccd1', when((col('ccd1') =='IND')| (col('ccd1') =='THA')| (col('ccd1') =='CHN')| (col('ccd1') =='AUS')| (col('ccd1') =='JPN')| (col('ccd1') =='HKG'),'APAC').otherwise(col('ccd1')))


features = features.withColumn('ccd1', when((col('ccd1') !='SCH')& (col('ccd1') !='CARR')& (col('ccd1') !='AAA')
                                                    & (col('ccd1') !='MEX')& (col('ccd1') !='CAN')& (col('ccd1') !='APAC')& (col('ccd1') !='CSP'),
                                                    'Others').otherwise(col('ccd1')))

In [ ]:
tvl1_call1_bucketizer = Bucketizer(splits=[float('-Inf'), 0,1,2,7,30,float('Inf') ],inputCol="tvl1_call1", outputCol="tvl1_call1_buckets")
bill_call1_bucketizer = Bucketizer(splits=[float('-Inf'), 0,1,2,7,30,float('Inf') ],inputCol="bill_call1", outputCol="bill_call1_buckets")
access_amt_bucketizer = Bucketizer(splits=[float('-Inf'), 0,10,50,100,150,float('Inf') ],inputCol="access_amt", outputCol="access_amt_buckets")
access_data_diff_bucketizer = Bucketizer(splits=[float('-Inf'),0,5,10,float('Inf') ],inputCol="access_data_diff", outputCol="access_data_diff_buckets")
feat_amt_bucketizer = Bucketizer(splits=[float('-Inf'), 0,10,20,100,float('Inf') ],inputCol="feat_amt", outputCol="feat_amt_buckets")
late_pymnt_chrg_amt_bucketizer = Bucketizer(splits=[float('-Inf'), 0,5,10,float('Inf') ],inputCol="late_pymnt_chrg_amt", outputCol="late_pymnt_chrg_amt_buckets")
occ_amt_bucketizer = Bucketizer(splits=[float('-Inf'), -10,0,10,float('Inf') ],inputCol="occ_amt", outputCol="occ_amt_buckets")
prev_tvl_count_bucketizer = Bucketizer(splits=[float('-Inf'), 0,1,2,3,5,10,float('Inf') ],inputCol="prev_tvl_count", outputCol="prev_tvl_count_buckets")
prev_call_count_bucketizer = Bucketizer(splits=[float('-Inf'), 0,1,2,3,5,10,float('Inf') ],inputCol="prev_call_count", outputCol="prev_call_count_buckets")

In [ ]:
features= tvl1_call1_bucketizer.setHandleInvalid("keep").transform(features)
features= bill_call1_bucketizer.setHandleInvalid("keep").transform(features)
features= access_amt_bucketizer.setHandleInvalid("keep").transform(features)
features= access_data_diff_bucketizer.setHandleInvalid("keep").transform(features)
features= feat_amt_bucketizer.setHandleInvalid("keep").transform(features)
features= late_pymnt_chrg_amt_bucketizer.setHandleInvalid("keep").transform(features)
features= occ_amt_bucketizer.setHandleInvalid("keep").transform(features)
features= prev_tvl_count_bucketizer.setHandleInvalid("keep").transform(features)
features= prev_call_count_bucketizer.setHandleInvalid("keep").transform(features)

In [ ]:
features = features.withColumn('primary1_topic', when(features['primary1_topic'].isNull(), 'Unknown').otherwise(col('primary1_topic')))
features = features.withColumn('primary1_topic', when((col('primary1_topic') =='FRAUD')| (col('primary1_topic') =='STOLEN & DAMAGED'),'PLAN & TRAVEL PASS').otherwise(col('primary1_topic')))
features = features.withColumn('primary1_topic', when(col('primary1_topic') =='PHONE SETTINGS','phone_settings').otherwise(col('primary1_topic')))
features = features.withColumn('primary1_topic', when(col('primary1_topic') =='BILLING','billing').otherwise(col('primary1_topic')))
features = features.withColumn('primary1_topic', when(col('primary1_topic') =='CRUISE','cruise').otherwise(col('primary1_topic')))
features = features.withColumn('primary1_topic', when(col('primary1_topic') =='PLAN & TRAVEL PASS','plan_travelpass').otherwise(col('primary1_topic')))

In [ ]:
#####Copied temporary primary1_topic to primary_topic
features=features.withColumn('primary_topic',col('primary1_topic'))
features = features.withColumn('primary_topic', when(col('primary_topic') =='Unknown','plan_travelpass').otherwise(col('primary_topic')))

In [ ]:
features = features.withColumn('primary2_topic', when(features['primary2_topic'].isNull(), 'Unknown').otherwise(col('primary2_topic')))
features = features.withColumn('primary2_topic', when((col('primary2_topic') =='FRAUD')| (col('primary2_topic') =='STOLEN & DAMAGED'),'PLAN & TRAVEL PASS').otherwise(col('primary2_topic')))
features = features.withColumn('primary2_topic', when(col('primary2_topic') =='PHONE SETTINGS','phone_settings').otherwise(col('primary2_topic')))
features = features.withColumn('primary2_topic', when(col('primary2_topic') =='BILLING','billing').otherwise(col('primary2_topic')))
features = features.withColumn('primary2_topic', when(col('primary2_topic') =='CRUISE','cruise').otherwise(col('primary2_topic')))
features = features.withColumn('primary2_topic', when(col('primary2_topic') =='PLAN & TRAVEL PASS','plan_travelpass').otherwise(col('primary2_topic')))

In [ ]:
features = features.withColumn('primary3_topic', when(features['primary3_topic'].isNull(), 'Unknown').otherwise(col('primary3_topic')))
features = features.withColumn('primary3_topic', when((col('primary3_topic') =='FRAUD')| (col('primary3_topic') =='STOLEN & DAMAGED'),'PLAN & TRAVEL PASS').otherwise(col('primary3_topic')))
features = features.withColumn('primary3_topic', when(col('primary3_topic') =='PHONE SETTINGS','phone_settings').otherwise(col('primary3_topic')))
features = features.withColumn('primary3_topic', when(col('primary3_topic') =='BILLING','billing').otherwise(col('primary3_topic')))
features = features.withColumn('primary3_topic', when(col('primary3_topic') =='CRUISE','cruise').otherwise(col('primary3_topic')))
features = features.withColumn('primary3_topic', when(col('primary3_topic') =='PLAN & TRAVEL PASS','plan_travelpass').otherwise(col('primary3_topic')))

In [ ]:
features = features.withColumn('primary4_topic', when(features['primary4_topic'].isNull(), 'Unknown').otherwise(col('primary4_topic')))
features = features.withColumn('primary4_topic', when((col('primary4_topic') =='FRAUD')| (col('primary4_topic') =='STOLEN & DAMAGED'),'PLAN & TRAVEL PASS').otherwise(col('primary4_topic')))
features = features.withColumn('primary4_topic', when(col('primary4_topic') =='PHONE SETTINGS','phone_settings').otherwise(col('primary4_topic')))
features = features.withColumn('primary4_topic', when(col('primary4_topic') =='BILLING','billing').otherwise(col('primary4_topic')))
features = features.withColumn('primary4_topic', when(col('primary4_topic') =='CRUISE','cruise').otherwise(col('primary4_topic')))
features = features.withColumn('primary4_topic', when(col('primary4_topic') =='PLAN & TRAVEL PASS','plan_travelpass').otherwise(col('primary4_topic')))

In [ ]:
features = features.withColumn('primary5_topic', when(features['primary5_topic'].isNull(), 'Unknown').otherwise(col('primary5_topic')))
features = features.withColumn('primary5_topic', when((col('primary5_topic') =='FRAUD')| (col('primary5_topic') =='STOLEN & DAMAGED'),'PLAN & TRAVEL PASS').otherwise(col('primary5_topic')))
features = features.withColumn('primary5_topic', when(col('primary5_topic') =='PHONE SETTINGS','phone_settings').otherwise(col('primary5_topic')))
features = features.withColumn('primary5_topic', when(col('primary5_topic') =='BILLING','billing').otherwise(col('primary5_topic')))
features = features.withColumn('primary5_topic', when(col('primary5_topic') =='CRUISE','cruise').otherwise(col('primary5_topic')))
features = features.withColumn('primary5_topic', when(col('primary5_topic') =='PLAN & TRAVEL PASS','plan_travelpass').otherwise(col('primary5_topic')))

In [ ]:
#features.dtypes

## Load data pipeline object which will perform pre-processing functions

In [ ]:
#In prodcution env. the path will be hdfs path where the object is stored
#from pyspark.ml import Pipeline, PipelineModel
#load_dataset_pipe = PipelineModel.load('/user/vjain01/final_sc_11_11_2019')
load_dataset_pipe = PipelineModel.load('/user/vjain01/Intl_caller_intent_pred_model/intl_call_intent_pred_mdl_datapipeline')

In [ ]:
testing_datapipe=load_dataset_pipe.transform(features)

## Load previously trained ML model object

## Predictions on new batch data based on trained model

In [ ]:
rfgModel=RandomForestClassificationModel.load('/user/vjain01/Intl_caller_intent_pred_model/intl_call_intent_pred_mdl_rf_object')

In [ ]:
predictions_all_labels = rfgModel.transform(testing_datapipe)

In [ ]:
predictions_all_labels.dtypes

In [ ]:
predictions_all_labels=predictions_all_labels.drop('primary_topic')

In [ ]:
predictions_all_labels.show(10)

In [ ]:
predictions_norm=predictions_all_labels.select('mtnconcat','normalised_probability','process_date','Prediction_Index','primary')

In [ ]:
predictions_norm.dtypes

In [ ]:
#predictions_norm.write.parquet("/user/kreddy01/Intl_caller_intent_pred_model/predictions_norm.parquet")

In [ ]:
predictions_norm.count()

In [ ]:
udf_object_0=udf(lambda s: np.around(np.array([np.amax(np.array(s))]),decimals=5).tolist(),ArrayType(DoubleType()))
udf_object_1=udf(lambda s: float(s[0]),FloatType())

In [ ]:
predictions_norm = predictions_norm.withColumn('max_probability',udf_object_1(udf_object_0('normalised_probability')))

In [ ]:
udf_object_0t=udf(lambda s: np.around(np.array([np.array(s)[0]]),decimals=5).tolist(),ArrayType(DoubleType()))
udf_object_1t=udf(lambda s: float(s[0]),FloatType())

In [ ]:
udf_object_0b=udf(lambda s: np.around(np.array([np.array(s)[1]]),decimals=5).tolist(),ArrayType(DoubleType()))
udf_object_1b=udf(lambda s: float(s[0]),FloatType())

In [ ]:
udf_object_0c=udf(lambda s: np.around(np.array([np.array(s)[2]]),decimals=5).tolist(),ArrayType(DoubleType()))
udf_object_1c=udf(lambda s: float(s[0]),FloatType())

In [ ]:
udf_object_0p=udf(lambda s: np.around(np.array([np.array(s)[3]]),decimals=5).tolist(),ArrayType(DoubleType()))
udf_object_1p=udf(lambda s: float(s[0]),FloatType())

In [ ]:
predictions_norm = predictions_norm.withColumn('tp',udf_object_1t(udf_object_0t('normalised_probability')))

In [ ]:
predictions_norm = predictions_norm.withColumn('b',udf_object_1b(udf_object_0b('normalised_probability')))

In [ ]:
predictions_norm = predictions_norm.withColumn('c',udf_object_1c(udf_object_0c('normalised_probability')))

In [ ]:
predictions_norm = predictions_norm.withColumn('p',udf_object_1p(udf_object_0p('normalised_probability')))

In [ ]:
predictions_norm.count()

In [ ]:
predictions_pandas=predictions_norm.toPandas()

In [ ]:
predictions_norm.dtypes

In [ ]:
from matplotlib.pyplot import plot as plt

In [ ]:
import numpy as np

In [ ]:
tp_hist,tp_bins=np.histogram(predictions_pandas['tp'],bins=100)
b_hist,b_bins=np.histogram(predictions_pandas['b'],bins=100)
c_hist,c_bins=np.histogram(predictions_pandas['c'],bins=100)
p_hist,p_bins=np.histogram(predictions_pandas['p'],bins=100)

In [ ]:
tp_cum=np.cumsum(tp_hist)
b_cum=np.cumsum(b_hist)
c_cum=np.cumsum(c_hist)
p_cum=np.cumsum(p_hist)

In [ ]:
import pandas as pd

In [ ]:
plt(tp_bins[:100],tp_cum,'b')
plt(b_bins[:100],b_cum,'r')
plt(c_bins[:100],c_cum,'g')
plt(p_bins[:100],p_cum,'y')

In [ ]:
predictions_pandas['pm']=predictions_pandas['p'].transform(lambda x : (0.34*x)+0.14 if (x>0.2 and x<0.5) else ((3.34*x)-1.52 if (x<0.65 and x>0.5) else x))

In [ ]:
tp_hist,tp_bins=np.histogram(predictions_pandas['tp'],bins=100)
b_hist,b_bins=np.histogram(predictions_pandas['b'],bins=100)
c_hist,c_bins=np.histogram(predictions_pandas['c'],bins=100)
p_hist,p_bins=np.histogram(predictions_pandas['p'],bins=100)
pm_hist,pm_bins=np.histogram(predictions_pandas['pm'],bins=100)

In [ ]:
tp_cum=np.cumsum(tp_hist)
b_cum=np.cumsum(b_hist)
c_cum=np.cumsum(c_hist)
p_cum=np.cumsum(p_hist)
pm_cum=np.cumsum(pm_hist)

In [ ]:
plt(tp_bins[:100],tp_cum,'b')
plt(b_bins[:100],b_cum,'r')
plt(c_bins[:100],c_cum,'g')
plt(p_bins[:100],p_cum,'y')
plt(pm_bins[:100],pm_cum,'k')

In [ ]:
from pandas.plotting import parallel_coordinates

In [ ]:
parallel_coordinates(predictions_pandas[['tp','b','c','p','pm','mod_originals']],'mod_originals',color=['w','w','b','w'],alpha=0.1)

In [ ]:
from pandas.plotting import radviz
radviz(predictions_pandas[['tp','b','c','p','mod_originals']],'mod_originals',color=['b','r','g','y'])

In [ ]:
predictions_pandas

In [ ]:
predictions_pandas['mod_strings'] = predictions_pandas[['tp','b','c','pm']].idxmax(axis=1)

In [ ]:
category = {'tp': 0.0,'b': 1.0,'c':2.0,'pm':3.0} 

In [ ]:
category_original= {'PLAN & TRAVEL PASS': 0.0,'BILLING': 1.0,'CRUISE':2.0,'PHONE SETTINGS':3.0,'FRAUD': 0.0,'STOLEN & DAMAGED': 0.0} 

In [ ]:
predictions_pandas['mod_predictions'] = [category[item] for item in predictions_pandas['mod_strings']]

In [ ]:
predictions_pandas['mod_originals'] = [category_original[item] for item in predictions_pandas['primary']]

In [ ]:
predictions_pandas

In [ ]:
predictions_pandas.dtypes

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix(predictions_pandas['mod_originals'], predictions_pandas['Prediction_Index'])

In [ ]:
confusion_matrix(predictions_pandas['mod_originals'], predictions_pandas['mod_predictions'])